# Parsing Dates

In [8]:
from cord.cord19 import *
from cord.core import parallel, ifnone
import pandas as pd
pd.options.display.max_colwidth = 120
pd.options.display.max_rows = 2000

In [7]:
research_papers = ResearchPapers.from_pickle()

In [6]:
research_papers['0015023cc06b5362d332b3baf348d11567ca2fbb']

,Value
sha,0015023cc06b5362d332b3baf348d11567ca2fbb
source_x,biorxiv
title,The RNA pseudoknots in foot-and-mouth disease virus are dispensable for genome replication but essential for the production of infectious virus.
doi,http://doi.org/10.1101/2020.01.10.901801
pmcid,
pubmed_id,
license,See https://www.biorxiv.org/about-biorxiv
abstract,The positive stranded RNA genomes of picornaviruses comprise a single large open reading frame flanked by 5' and 3' untranslated regions (UTRs). Foot-and-mouth disease virus (FMDV) has an unusuall...
authors,"Ward, J. C. J.; Lasecka-Dykes, L.; Neil, C.; Adeyemi, O.; Gold, S.; McLean, N.; Wright, C.; Herod, M. R.; Kealy, D.; Warner, E.; King, D. P.; Tuthill, T. J.; Rowlands, D. J.; Stonehouse, N. J."
journal,


In [5]:
research_papers.save()

Saving to data\ResearchPapers.pickle
